In [1]:
from tqdm import tqdm
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
PATH_TRAIN = "datasets/train.csv"
PATH_VALID = "datasets/valid.csv"

In [3]:
data_train = pd.read_csv(PATH_TRAIN)
data_valid = pd.read_csv(PATH_VALID)

# Separate features (X) and target variable (y)
y_train = data_train['Pret']
X_train = data_train.drop('Pret', axis=1)

y_valid = data_valid['Pret']
X_valid = data_valid.drop('Pret', axis=1)

print(X_train.head())

                     Versiune  Vechime      Km  Putere  Capacitate cilindrica  \
0          D4 AWD Inscription      6.0  124000     190                   1969   
1      109 CDI Extralung Mixt      5.0   36000      90                   1461   
2  1.6 BlueHDi FAP STT Active      8.0  219000     120                   1560   
3      2.2 CRDi 2WD Signature      4.0   51500     202                   2151   
4                  1.8 TSI FR     11.0  228300     160                   1798   

           Marca     Model Combustibil     Transmisie Norma de poluare  \
0          Volvo     XC 60      Diesel  4x4 (automat)           Euro 6   
1  Mercedes-Benz     Citan      Diesel           Fata           Euro 6   
2        Peugeot       308      Diesel           Fata           Euro 6   
3        Hyundai  Santa Fe      Diesel           Fata           Euro 6   
4           Seat      Leon     Benzina           Fata           Euro 5   

  Cutie de viteze Tip Caroserie Optiuni culoare Tara de origine     

In [4]:
numeric_features = ["Km", "Vechime", "Capacitate cilindrica", "Putere"]
categorical_features = ["Marca", "Model", "Versiune", "Combustibil", "Transmisie", "Norma de poluare", "Tip Caroserie", "Cutie de viteze", "Stare", "Tara de origine", "Optiuni culoare"]

numeric_features_KNN = ["Km", "Vechime", "Putere"]
categorical_features_KNN = ["Marca", "Model", "Versiune", "Combustibil"]


In [5]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

preprocessor_KNN = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_KNN),
        ('cat', categorical_transformer, categorical_features_KNN)
    ])

In [6]:
regressor1 = SVR(C = 5000000, kernel = 'rbf')

regressor2 = KNeighborsRegressor(n_neighbors=1)

model1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor1)
])

model2 = Pipeline(steps=[
    ('preprocessor', preprocessor_KNN),
    ('regressor', regressor2)
])

model1.fit(X_train, y_train)
model2.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Km', 'Vechime', 'Putere']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Marca', 'Model', 'V

In [7]:
predictions = model1.predict(X_valid)
predictions2 = model2.predict(X_valid)

for i in range(len(predictions)):
    predictions[i] = (predictions[i] * 0.52) + (predictions2[i] * 0.48)

mae = mean_absolute_error(y_valid, predictions)
print("The MAE score of the model on validation data is: " + str(mae))

rmse = mean_squared_error(y_valid, predictions, squared=False)
print("The RMSE score of the model on validation data is: " + str(rmse))

The MAE score of the model on validation data is: 1329.610791491743
The RMSE score of the model on validation data is: 8718.526220829184
